In [ ]:
# Verify we're in the correct environment
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

# Check if we can import the required modules
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    print(f"✅ CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✅ CUDA device count: {torch.cuda.device_count()}")
        print(f"✅ Current CUDA device: {torch.cuda.current_device()}")
except ImportError as e:
    print(f"❌ PyTorch import failed: {e}")

try:
    import imageio
    print(f"✅ imageio version: {imageio.__version__}")
except ImportError as e:
    print(f"❌ imageio import failed: {e}")

try:
    import numpy as np
    import matplotlib.pyplot as plt
    print(f"✅ numpy version: {np.__version__}")
    print(f"✅ matplotlib imported successfully")
except ImportError as e:
    print(f"❌ numpy/matplotlib import failed: {e}")


In [ ]:
import os
import sys

# Set up paths
nerf_path = '/home/ubuntu/mono_to_3d/neural_radiance_fields/open_source_implementations/nerf-pytorch'
print(f"NeRF path: {nerf_path}")
print(f"Path exists: {os.path.exists(nerf_path)}")

# Change to NeRF directory and add to Python path
os.chdir(nerf_path)
if nerf_path not in sys.path:
    sys.path.insert(0, nerf_path)

print(f"Current working directory: {os.getcwd()}")
print(f"Directory contents: {os.listdir('.')[:10]}")
print(f"Data directory exists: {os.path.exists('./data')}")


In [ ]:
# Import NeRF modules
try:
    from load_blender import load_blender_data
    print("✅ NeRF modules imported successfully!")
except ImportError as e:
    print(f"❌ NeRF module import failed: {e}")
    import traceback
    traceback.print_exc()

# Load the LEGO dataset
try:
    print("Loading LEGO dataset...")
    images, poses, render_poses, hwf, i_split = load_blender_data('./data/nerf_synthetic/lego', half_res=True, testskip=1)
    
    print(f"✅ Data loaded successfully!")
    print(f"Images shape: {images.shape}")
    print(f"Poses shape: {poses.shape}")
    print(f"Render poses shape: {render_poses.shape}")
    print(f"HWF (Height, Width, Focal): {hwf}")
    print(f"Train/Val/Test split: {[len(split) for split in i_split]}")
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# Display sample images from the dataset
if 'images' in locals():
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for i in range(min(8, len(images))):
        axes[i].imshow(images[i])
        axes[i].set_title(f"Image {i}")
        axes[i].axis('off')
    
    plt.suptitle("Sample LEGO Images from NeRF Dataset", fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Show image statistics
    print(f"Image data type: {images.dtype}")
    print(f"Image value range: [{images.min():.3f}, {images.max():.3f}]")
    print(f"Image shape (single): {images[0].shape}")
else:
    print("❌ Images not loaded - cannot display visualizations")


In [ ]:
# Run a very short NeRF training test (just a few iterations)
import subprocess

print("Running quick NeRF training test (10 iterations)...")
print("This may take a few minutes...")

# Use the quick test script that modifies the iteration count
cmd = """
source ~/miniconda3/etc/profile.d/conda.sh && 
conda activate nerf_env && 
python /home/ubuntu/mono_to_3d/neural_radiance_fields/quick_test_nerf.py
"""

try:
    result = subprocess.run(['bash', '-c', cmd], capture_output=True, text=True, timeout=300)
    
    print("=== TRAINING OUTPUT ===")
    if result.stdout:
        # Show last 2000 characters to avoid overwhelming output
        print(result.stdout[-2000:])
    
    if result.stderr:
        print("=== ERRORS/WARNINGS ===")
        print(result.stderr[-1000:])
    
    if result.returncode == 0:
        print("✅ NeRF training test completed successfully!")
    else:
        print(f"❌ NeRF training failed with return code: {result.returncode}")
        print("💡 Try running the data loading test first to verify setup")
        
except subprocess.TimeoutExpired:
    print("⏰ Training test timed out (5 minutes) - this is normal for longer runs")
except Exception as e:
    print(f"❌ Error running NeRF training: {e}")

print("\n" + "="*50)
print("💡 ALTERNATIVE: Test data loading only")
print("="*50)

# Alternative: Just test data loading without training
try:
    print("Testing data loading...")
    from load_blender import load_blender_data
    
    images, poses, render_poses, hwf, i_split = load_blender_data('./data/nerf_synthetic/lego', half_res=True, testskip=1)
    
    print(f"✅ Data loading test successful!")
    print(f"Images shape: {images.shape}")
    print(f"Poses shape: {poses.shape}")
    print(f"Splits: train={len(i_split[0])}, val={len(i_split[1])}, test={len(i_split[2])}")
    print("\n🎯 NeRF environment is properly configured!")
    
except Exception as e:
    print(f"❌ Data loading failed: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
print("=== NeRF DEMO SUMMARY ===")
print("")
print("✅ Environment setup: Complete")
print("✅ Data loading: Complete" if 'images' in locals() else "❌ Data loading: Failed")
print("✅ Visualization: Complete" if 'images' in locals() else "❌ Visualization: Skipped")
print("")
print("=== NEXT STEPS ===")
print("1. For full training: python run_nerf.py --config configs/lego.txt")
print("2. For rendering only: python run_nerf.py --config configs/lego.txt --render_only")
print("3. Available datasets: lego, fern, chair, drums, hotdog, materials, mic, ship")
print("4. Training typically takes 4-8 hours on GPU for good results")
print("")
print("=== AVAILABLE COMMANDS ===")
print("# Quick test (10 iterations):")
print("python run_nerf.py --config configs/lego.txt --N_iters 10")
print("")
print("# Full training:")
print("python run_nerf.py --config configs/lego.txt")
print("")
print("# Render test views:")
print("python run_nerf.py --config configs/lego.txt --render_only --render_test")
